# Introducción a Vertex AI Gemini API & Python SDK

## Información General

### Gemini

Gemini es una familia de modelos de IA generativa desarrollados por Google DeepMind que está diseñada para casos de uso multimodal. La API de Gemini te da acceso a los modelos Gemini Pro y Gemini Pro Vision.

### API de Vertex AI Gemini

La API de Vertex AI Gemini proporciona una interfaz unificada para interactuar con los modelos Gemini. Hay dos modelos Gemini 1.0 Pro disponibles en la API de Gemini:

- **Modelo Gemini 1.0 Pro** (`gemini-1.0-pro`): Diseñado para manejar tareas de lenguaje natural, chat de texto y código de múltiples turnos, y generación de código.
- **Modelo Gemini 1.0 Pro Vision** (`gemini-1.0-pro-vision`): Soporta prompts multimodales. Puedes incluir texto, imágenes y video en tus solicitudes de prompts y obtener respuestas de texto o código.
- **Modelo Gemini 1.5 Pro** (`gemini-1.5-pro-preview-0409`): Soporta prompts multimodales. Puedes incluir texto, imágenes, video y pdf en tus solicitudes de prompts y obtener respuestas de texto o código.

Puedes interactuar con la API de Gemini utilizando los siguientes métodos:
#
- Usa [Vertex AI Studio](https://cloud.google.com/generative-ai-studio) para pruebas rápidas y generación de comandos
- Usa comandos cURL
- Usa el SDK de Vertex AI

Este cuaderno se centra en usar el **SDK de Vertex AI para Python** para llamar a la API de Vertex AI Gemini.

Para más información, consulta la documentación de [IA generativa en Vertex AI](https://cloud.google.com/vertex-ai/docs/generative-ai/learn/overview).


### Objetivos

En este tutorial, aprenderás a usar la API de Vertex AI Gemini con el SDK de Vertex AI para Python para interactuar con el modelo Gemini 1.0 Pro (`gemini-1.0-pro`) y el modelo Gemini 1.0 Pro Vision (`gemini-1.0-pro-vision`).

Completarás las siguientes tareas:

- Instalar el SDK de Vertex AI para Python
- Usar la API de Vertex AI Gemini para interactuar con cada modelo
  - Modelo Gemini 1.0 Pro (`gemini-1.0-pro`):
    - Generar texto a partir de prompts de texto
    - Explorar diversas características y opciones de configuración
  - Modelo Gemini 1.0 Pro Vision (`gemini-1.0-pro-vision`):
    - Generar texto a partir de imágenes y prompts de texto
    - Generar texto a partir de video y prompts de texto


## Comenzando


### Instalar Vertex AI SDK para Python


In [ ]:
! pip3 install --upgrade --user google-cloud-aiplatform

### Reiniciar el entorno de ejecución actual (si están en Jupyter)

Para usar los paquetes recién instalados en este entorno de ejecución de Jupyter, debes reiniciar el entorno. Puedes hacerlo ejecutando la celda a continuación, lo que reiniciará el kernel actual.


In [ ]:
# Reinicia el kernel después de la instalación para que el entorno pueda acceder a los nuevos packetes.
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

### Autenticate (solo Colab)

Si estás ejecutando este cuaderno en Google Colab, ejecuta la siguiente celda para autenticar tu entorno. Este paso no es necesario si estás utilizando [Vertex AI Workbench](https://cloud.google.com/vertex-ai-workbench).


In [1]:
import sys

# Autenticación adicional es requerida para Google Colab
if "google.colab" in sys.modules:
    # Autentica el usuario en Google Cloud
    from google.colab import auth

    auth.authenticate_user()

### Establecer la información del proyecto de Google Cloud e inicializar el SDK de Vertex AI

Para comenzar a usar Vertex AI, debes tener un proyecto de Google Cloud existente y [habilitar la API de Vertex AI](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com).

Aprende más sobre [cómo configurar un proyecto y un entorno de desarrollo](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).


In [2]:
# Define la información del proyecto
PROJECT_ID = "prj-uc-genai-labs"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

# Inicializar Vertex AI
import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION)

### Importar librerías


In [3]:
from vertexai.generative_models import GenerationConfig, GenerativeModel, Image, Part

## Utilizar el modelo Gemini 1.0 Pro

El modelo Gemini 1.0 Pro (`gemini-1.0-pro`) está diseñado para manejar tareas de lenguaje natural, chat de texto y código de múltiples turnos, y generación de código.


### Cargar el modelo Gemini 1.0 Pro


In [4]:
model = GenerativeModel("gemini-1.0-pro")

Generar texto a partir de prompts de texto

Envía una prompt de texto al modelo. El modelo Gemini 1.0 Pro (`gemini-1.0-pro`) proporciona un mecanismo de streaming. Con este modo, no necesitas esperar la respuesta completa; puedes comenzar a procesar fragmentos tan pronto como estén accesibles.


In [ ]:
responses = model.generate_content("Por qué es el cielo azul?", stream=True)

for response in responses:
    print(response.text, end="")

#### Prueba tus propios prompts

- ¿Cuáles son los mayores desafíos que enfrenta la industria de la salud?
- ¿Cuáles son los últimos desarrollos en la industria automotriz?
- ¿Cuáles son las mayores oportunidades en la industria minorista?
- (¡Prueba tus propios prompts!)


In [ ]:
prompt = """Crea una lista numerada de 10 elementos. Cada elemento en la lista debe ser una tendencia en la industria tecnológica.

Cada tendencia debe ser de menos de 5 palabras."""  # intenta tu propio prompt

responses = model.generate_content(prompt, stream=True)

for response in responses:
    print(response.text, end="")

#### Parámetros del modelo

Cada prompt que envías al modelo incluye valores de parámetros que controlan cómo el modelo genera una respuesta. El modelo puede generar diferentes resultados para diferentes valores de parámetros. Puedes experimentar con diferentes parámetros del modelo para ver cómo cambian los resultados.


In [ ]:
generation_config = GenerationConfig(
    temperature=0.9,
    top_p=1.0,
    top_k=32,
    candidate_count=1,
    max_output_tokens=8192,
)

responses = model.generate_content(
    "Por qué es el cielo azul?",
    generation_config=generation_config,
    stream=True,
)

for response in responses:
    print(response.text, end="")

### Pruebas de prompts de chat

El modelo Gemini 1.0 Pro soporta conversaciones naturales de múltiples turnos y es ideal para tareas de texto que requieren interacciones de ida y vuelta. Los siguientes ejemplos muestran cómo responde el modelo durante una conversación de múltiples turnos.


In [ ]:
chat = model.start_chat()

prompt = """Mi nombre es Pablo. Eres mi asistente personal. Mis películas favoritas son El Señor de los Anillos y El Hobbit.

Sugiere otra película que podría gustarme.
"""

responses = chat.send_message(prompt, stream=True)

for response in responses:
    print(response.text, end="")

Este prompt de seguimiento muestra cómo responde el modelo basado en el prompt anterior:


In [ ]:
prompt = "¿Mis películas favoritas están basadas en una serie de libros?"
responses = chat.send_message(prompt, stream=True)

for response in responses:
    print(response.text, end="")

También puedes ver el historial de chat:


In [ ]:
print(chat.history)

## Usando el modelo Gemini 1.0 Pro Vision

Gemini 1.0 Pro Vision (`gemini-1.0-pro-vision`) es un modelo multimodal que soporta prompts multimodales. Puedes incluir texto, imagenes y video en tus prompts y obtener respuestas de texto o código.


### Cargar el modelo Gemini 1.0 Pro Vision


In [11]:
multimodal_model = GenerativeModel("gemini-1.0-pro-vision")

### Definir funciones de ayuda

Definir funciones de ayuda para cargar y mostrar imágenes.


In [12]:
import http.client
import typing
import urllib.request

import IPython.display
from PIL import Image as PIL_Image
from PIL import ImageOps as PIL_ImageOps


def display_images(
    images: typing.Iterable[Image],
    max_width: int = 600,
    max_height: int = 350,
) -> None:
    for image in images:
        pil_image = typing.cast(PIL_Image.Image, image._pil_image)
        if pil_image.mode != "RGB":
            # RGB es soportado por todos los entornos Jupyter (e.g. RGBA no todavía)
            pil_image = pil_image.convert("RGB")
        image_width, image_height = pil_image.size
        if max_width < image_width or max_height < image_height:
            # Reduce el tamaño para mostrarla más pequeña
            pil_image = PIL_ImageOps.contain(pil_image, (max_width, max_height))
        IPython.display.display(pil_image)


def get_image_bytes_from_url(image_url: str) -> bytes:
    with urllib.request.urlopen(image_url) as response:
        response = typing.cast(http.client.HTTPResponse, response)
        image_bytes = response.read()
    return image_bytes


def load_image_from_url(image_url: str) -> Image:
    image_bytes = get_image_bytes_from_url(image_url)
    return Image.from_bytes(image_bytes)


def get_url_from_gcs(gcs_uri: str) -> str:
    # convierte gcs uri a url para mostrar la imagen.
    url = "https://storage.googleapis.com/" + gcs_uri.replace("gs://", "").replace(
        " ", "%20"
    )
    return url


def print_multimodal_prompt(contents: list):
    """
    Given contents that would be sent to Gemini,
    output the full multimodal prompt for ease of readability.
    """
    for content in contents:
        if isinstance(content, Image):
            display_images([content])
        elif isinstance(content, Part):
            url = get_url_from_gcs(content.file_data.file_uri)
            IPython.display.display(load_image_from_url(url))
        else:
            print(content)

Generar texto a partir de imagen local y texto

Utiliza el método `Image.load_from_file` para cargar un archivo local como la imagen para generar texto.


In [ ]:
# Descarga una imagen de Google Cloud Storage
! gsutil cp "gs://cloud-samples-data/generative-ai/image/320px-Felis_catus-cat_on_snow.jpg" ./image.jpg

# Cargala de un archivo local
image = Image.load_from_file("image.jpg")

# Prepara el contenido
prompt = "Describe esta imagen?"
contents = [image, prompt]

responses = multimodal_model.generate_content(contents, stream=True)

print("-------Prompt--------")
print_multimodal_prompt(contents)

print("\n-------Response--------")
for response in responses:
    print(response.text, end="")

Generar texto a partir de texto e imagen(es)


#### Imágenes con URIs de Cloud Storage

Si tus imágenes están almacenadas en [Cloud Storage](https://cloud.google.com/storage/docs), puedes especificar el URI de Cloud Storage de la imagen para incluirla en el prompt. También debes especificar el campo `mime_type`. Los tipos MIME soportados para imágenes incluyen `image/png` y `image/jpeg`.

Nota que el URI (no confundir con URL) para un objeto de Cloud Storage siempre debe comenzar con `gs://`.


In [ ]:
# Carga una imagen desde Cloud Storage URI
gcs_uri = "gs://cloud-samples-data/generative-ai/image/boats.jpeg"

# Prepara el contenido
image = Part.from_uri(gcs_uri, mime_type="image/jpeg")
prompt = "Describe esta escena?"
contents = [image, prompt]

responses = multimodal_model.generate_content(contents, stream=True)

print("-------Prompt--------")
print_multimodal_prompt(contents)

print("\n-------Response--------")
for response in responses:
    print(response.text, end="")

Imágenes con links

También puedes utilizar links a imágenes, como se muestra a continuación. La función de ayuda `load_image_from_url()` (que se declaró anteriormente) convierte la imagen a bytes y la devuelve como un objeto Image que luego puede enviarse a Gemini 1.0 Pro Vision junto con el texto del prompt.


In [ ]:
# Carga una imagen desde Cloud Storage URI
image_url = (
    "https://storage.googleapis.com/cloud-samples-data/generative-ai/image/boats.jpeg" # En este caso el bucket es público
)
image = load_image_from_url(image_url)  # convert to bytes

# Prepara el contenido
prompt = "Describe la escena"
contents = [image, prompt]

responses = multimodal_model.generate_content(contents, stream=True)

print("-------Prompt--------")
print_multimodal_prompt(contents)

print("\n-------Response--------")
for response in responses:
    print(response.text, end="")

#### Combinando múltiples imágenes y textos en el prompt para hacer few-shot prompting


Puedes enviar más de una imagen a la vez, y también colocar tus imágenes en cualquier lugar junto a tu texto de prompt.

En el ejemplo a continuación, se realiza un prompting de few-shot para que Gemini 1.0 Pro Vision devuelva la ciudad y el monumento en un formato JSON específico.


In [ ]:
# Carga imagenes desde Cloud Storage URI
image1_url = "https://storage.googleapis.com/github-repo/img/gemini/intro/landmark1.jpg"
image2_url = "https://storage.googleapis.com/github-repo/img/gemini/intro/landmark2.jpg"
image3_url = "https://storage.googleapis.com/github-repo/img/gemini/intro/landmark3.jpg"
image1 = load_image_from_url(image1_url)
image2 = load_image_from_url(image2_url)
image3 = load_image_from_url(image3_url)

# Prepara los prompts
prompt1 = """{"city": "London", "Landmark:", "Big Ben"}"""
prompt2 = """{"city": "Paris", "Landmark:", "Eiffel Tower"}"""

# Prepara el contenido
contents = [image1, prompt1, image2, prompt2, image3]

responses = multimodal_model.generate_content(contents, stream=True)

print("-------Prompt--------")
print_multimodal_prompt(contents)

print("\n-------Response--------")
for response in responses:
    print(response.text, end="")

Generar texto a partir de un archivo de video

Especifica el URI de Cloud Storage del video para incluirlo en el prompt. El bucket que almacena el archivo debe estar en el mismo proyecto de Google Cloud que envía la solicitud. También debes especificar el campo `mime_type`. El tipo MIME soportado para video incluye `video/mp4`.


In [ ]:
file_path = "github-repo/img/gemini/multimodality_usecases_overview/pixel8.mp4"
video_uri = f"gs://{file_path}"
video_url = f"https://storage.googleapis.com/{file_path}"

IPython.display.Video(video_url, width=450)

In [ ]:
prompt = """
Responde las siguientes preguntas usando únicamente el video:
¿Cuál es la profesión de la persona principal?
¿Cuáles son las características principales del teléfono destacadas?
¿En qué ciudad se grabó esto?
Proporciona la respuesta en JSON.
"""

video = Part.from_uri(video_uri, mime_type="video/mp4")
contents = [prompt, video]

responses = multimodal_model.generate_content(contents, stream=True)

for response in responses:
    print(response.text, end="")